# Imports 

In [ ]:
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
from jax import grad
import jax
import numpy as np
%matplotlib inline
from grad import *
from model import *

# Test of my autodiff library 

In [14]:
'''
This bit deals with how the computation graph is managed. The two outputs should be the same.

Here, Number is my from-near-scratch autodiff library, kind of like how torch.tensor() is
'''
x1 = Number(1)
x2 = Number(2)
x3 = Number(3)
x4 = Number(4)
x5 = Number(5)

y = (x1+x2)*x5 - x3
top_sorted = topo_sort(y)

print("Topologically sorted Values: ", top_sorted)

y.null_gradients()

for num in top_sorted:
    num.backprop_single()
    
print("Backprop with topological sort", [(i.grad, i) for i in top_sorted])

y.null_gradients()
y.backprop()

print("Backprop from the value", [(i.grad, i) for i in top_sorted])

Topologically sorted Values:  [Number(12), Number(3), Number(15), Number(5), Number(3), Number(2), Number(1)]
Backprop with topological sort [(1, Number(12)), (-1, Number(3)), (1, Number(15)), (3, Number(5)), (5, Number(3)), (5, Number(2)), (5, Number(1))]
Backprop from the value [(1, Number(12)), (-1, Number(3)), (1, Number(15)), (3, Number(5)), (5, Number(3)), (5, Number(2)), (5, Number(1))]


In [17]:
'''
For the next 3 cells, me and jax compare values on functions critical to neural networks.
'''

testmine = Number(2.)
mysigmoid = 1/(1+math.e**-testmine)

mysigmoid.backprop(should_print=False)

def jaxsigmoidsum(x):
    x = jnp.sum(x)
    return 1 / (1 + jnp.exp(-x)) 

testjax = jnp.array([2.]) 
sigmoided_value, grads = jax.value_and_grad(jaxsigmoidsum, argnums=(0))(testjax)

print(f"value comparison:", f"Mine {mysigmoid}", f"Jax {sigmoided_value}")
print(f"Grad comparison:", f"Mine {[testmine.grad]}", f"Jax{grads}")

value comparison: Mine Number(0.8807970779778823) Jax 0.8807970285415649
Grad comparison: Mine [0.1049935854035065] Jax[0.10499357]


In [19]:
def jax_sigmoid(x):
    return jnp.vectorize(lambda x: 1/(1+math.e**-x))(x)

def jax_weight_matrix(shape, naive=False):
    """weight matrix thingy.give dims. Not 0."""
    number = 1
    if(type(shape) == int):
        shape = [shape]
    for i in shape:
        number*= i
    if naive:
        return jnp.array([(i / 10) for i in range(number)]).reshape(*shape)
    return np.array([np.random.uniform(low=-.2, high=.2, size=None) for i in range(number)]).reshape(*shape)
    # return np.array([variable(np.random.uniform(low=-.2, high=.2, size=None)) for i in range(sizes[0] * sizes[1])).reshape(*shape)

In [21]:
test_shape = (3,5)
test_jax = jax_weight_matrix(test_shape, naive=True)
test_mine = weight_matrix(test_shape, naive=True)
test_shape2 = (5, 2)
test_jax2 = jax_weight_matrix(test_shape2, naive=True)
test_mine2 = weight_matrix(test_shape2, naive=True)

my_matmul = np.sum(test_mine @ test_mine2)
def j_matmul(a, b):
    thing = a @ b
    return jnp.sum(thing)


print("My value" , my_matmul)
print("Jax's value", j_matmul(test_jax, test_jax2))

j_matmuled, grads = jax.value_and_grad(j_matmul, argnums=(0, 1))(test_jax, test_jax2)

my_matmul.backprop()

print("Jax's grads" , grads[1].flatten())
print("My grads", [thing.grad for thing in test_mine2.flat])
#These match almost exactly! Yay! Please don't ask about the formatting

My value Number(10.65)
Jax's value 10.65
Jax's grads [1.5       1.5       1.8000001 1.8000001 2.1       2.1       2.4
 2.4       2.6999998 2.6999998]
My grads [1.5, 1.5, 1.8, 1.8, 2.0999999999999996, 2.0999999999999996, 2.4000000000000004, 2.4000000000000004, 2.7, 2.7]


# Overfitting a single image

In [2]:
import tensorflow as tf
import keras
%load_ext autoreload
%autoreload 2
import numpy as np

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(path="mnist.npz", )
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

def batch(x, y, batch_size=32):
    if len(x) % batch_size != 0:
        x = x[:batch_size * (len(x)//batch_size)]
        y=y[:batch_size*(len(x)//batch_size)]
    return np.array_split(x, len(x) / batch_size, axis=0), np.array_split(y, len(y)/batch_size, axis=0)

def fix_data(x, y):
    x = x.reshape(x.shape[0], 28*28)/255
    test = np.zeros((x.shape[0], 10))
    test[np.arange(x.shape[0]),y] = 1
    return (x, test)

fixed_x, fixed_y = fix_data(x_train[:1], y_train[:1]) #We take only one image
b_x , b_y = batch(fixed_x, fixed_y, 1)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

my_model = Model(28*28, 10, [ 8, 16])
datas = []
for _epoch in range(10):
    print(f"starting epoch {_epoch}")
    datas.append(my_model.train_epoch(b_x, b_y, lr=.1, timer=False, batch_timer=False))
#as you can see loss does go down and it manages to predict the single image.
#I tried really hard to get it to be able to fully train on all images, but it takes a very long time (in the hours+). 
#Intended behavior: At first, the acc (accuracy) is 0 because it has no idea what it's doing. But over time, it fits to that image and predicts that image (Acc of 1). Yay!
#Loss also does go down.

starting epoch 0
Acc: 0.0 Avg loss: 2.5404272913700097
Elapsed time for one epoch: 0.09765959996730089 seconds
starting epoch 1
Acc: 0.0 Avg loss: 2.4178759463385653
Elapsed time for one epoch: 0.1118340000975877 seconds
starting epoch 2
Acc: 0.0 Avg loss: 2.3027330912420005
Elapsed time for one epoch: 0.10104560013860464 seconds
starting epoch 3
Acc: 1.0 Avg loss: 2.1900675339014
Elapsed time for one epoch: 0.3303846998605877 seconds
starting epoch 4
Acc: 1.0 Avg loss: 2.0775970999406512
Elapsed time for one epoch: 0.10056080017238855 seconds
starting epoch 5
Acc: 1.0 Avg loss: 1.9638569088340436
Elapsed time for one epoch: 0.1067631000187248 seconds
starting epoch 6
Acc: 1.0 Avg loss: 1.8481116547506435
Elapsed time for one epoch: 0.32172450004145503 seconds
starting epoch 7
Acc: 1.0 Avg loss: 1.7302941843173607
Elapsed time for one epoch: 0.09617669996805489 seconds
starting epoch 8
Acc: 1.0 Avg loss: 1.610939576989026
Elapsed time for one epoch: 0.1012807001825422 seconds
starting 

In [ ]:
#Code to display img

# Attempt to actually train it on multiple images

For reasons that are embedded in my terrible architecture decisions, this is very long. (These reasons include the topo sort not being cached, which would be fairly difficult to implement due to how Numbers() are created. I did not make this homemade autodiff library for speed or even to truly train something; I made it to understand autodiff, which I think it has suceeded in doing, as demonstrated by above cells.)

Running any of the cells below may result in it taking over half an hour to an hour to truly train.

In [36]:
full_x, full_y = fix_data(x_train[:10000], y_train[:10000])
full_b_x , full_b_y = batch(fixed_x, fixed_y)

my_model = Model(28*28, 10, [ 8, 16])
datas = []
for _epoch in range(30):
    print(f"starting epoch {_epoch}")
    datas.append(my_model.train_epoch(b_x, b_y, lr=1e-2, timer=False, batch_timer=False))

312.0


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

fig = plt.figure(figsize=(10, 7))
pic = 1
for i, img in enumerate(x_test[:10]):
  plt.subplot(2, 5, pic)
  plt.axis('off')
  predicted = my_model.fd(img.flat)
  plt.title(f"T {y_test[i]} mine {np.argmax(predicted)} ")
  plt.imshow(img)
  pic+= 1
plt.show()
#60% acc. Considering this is from nearly scratch not terrible 